# 完整模型训练过程

### 准备数据集

In [1]:
import torchvision
from torch.utils.data import DataLoader
from torch import nn
from model import *
from torch.utils.tensorboard import SummaryWriter

train_data = torchvision.datasets.CIFAR10(root="../dataset", train=True, transform=torchvision.transforms.ToTensor(),download=True)
test_data = torchvision.datasets.CIFAR10(root="../dataset", train=False, transform=torchvision.transforms.ToTensor(),download=True)

train_data_size = len(train_data)
test_data_size = len(test_data)

print("训练集长度为{}".format(train_data_size))
print("测试集长度为{}".format(test_data_size))

Files already downloaded and verified
Files already downloaded and verified
训练集长度为50000
测试集长度为10000


### 数据预处理

In [2]:
# 准备数据集
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

### 创建网络实例，定义损失函数，优化器

In [3]:
# 运行的设备：pytorch==1.4.0  + rtx3080 条件下，只能用CPU跑，3080只支持cuda11以上的， 1.4只支持cuda10
device = torch.device("cpu")                   # 单显卡写 "cuda"或者"cuda:0"都是一样的

#  创建网络实例
tudui = Tudui()
tudui = tudui.to(device)

# 损失函数
loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.to(device)

# 优化器
learning_rate = 0.01
optimizer = torch.optim.SGD(tudui.parameters(), lr=learning_rate)

### 设置训练网络的一些参数

In [4]:
# 训练次数
total_train_step = 0
# 测试次数
total_test_step = 10
# epoch
epoch = 10

# 添加tensorboard
writer = SummaryWriter("./logs_train")

### 训练开始：   
**训练一个epoch  =  在训练集上完整学一次 + 在测试集上完整验证一次**   
这个地方有点尴尬，老版本的pytorch1.4支持的cuda版本我的显卡3080用不了.... 
所以这个地方只能用cpu的环境来跑，不过我在当前文件夹目录下另外保存了一个.py的文件，用pycharm配yolov5的解释器就能跑GPU版本的了。

In [6]:
for i in range(epoch):
    print("-----------------第{}轮训练开始------------------".format(i + 1))
    
    # 训练步骤开始
    tudui.train()
    for data in train_dataloader:
        imgs, targets = data                                                      # 从dataloader取出数据
        imgs = imgs.to(device)
        targets = targets.to(device)
        outputs = tudui(imgs)                                                 #  神经网络前向推理
        loss = loss_fn(input=outputs, target=targets)  #  计算真实值与目标之间的误差
        
        # 优化器调整模型参数
        optimizer.zero_grad()          # 将上次存留的梯度信息清零
        loss.backward()                      #   反向传播
        optimizer.step()                      #   修改权重参数
        
        # 打印训练结果
        total_train_step += 1
        if total_train_step % 100 == 0:              # 每训练100次打印一次结果
            print("训练次数:{} , Loss: {}".format(total_train_step, loss))
            writer.add_scalar("Train Loss", loss.item(), total_train_step)

    # 测试步骤开始：validation
    tudui.eval()
    total_accuracy = 0
    total_test_loss = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs, targets = data
            outputs = tudui(imgs)
            loss = loss_fn(input=outputs, target=targets)
            total_test_loss += loss
            accuracy = (outputs.argmax(1)  == targets).sum()                                                                      # 逐行进行比较
            # print(accuracy)
            total_accuracy += accuracy.item()
            # print("total is ："+str(total_accuracy))
    
    writer.add_scalar("Validation Loss", total_test_loss, total_test_step)
    print("本epoch上整体验证集上的Loss:  {}".format(total_test_loss))
    writer.add_scalar("Validation Acc  ", total_accuracy/test_data_size, total_test_step)
    print("本epoch上整体验证集上的Acc  :  {}".format(total_accuracy/test_data_size))
    
    # 本epoch的测试结束，计步器++
    total_test_step += 1

torch.save(tudui, "CAFIR10_ckpt_e10.pth")
print("Model has been saved")
writer.close()

-----------------第1轮训练开始------------------
训练次数:2500 , Loss: 1.659647822380066
训练次数:2600 , Loss: 1.4574962854385376
训练次数:2700 , Loss: 1.4238028526306152
训练次数:2800 , Loss: 1.6770062446594238
训练次数:2900 , Loss: 1.650638461112976
训练次数:3000 , Loss: 1.6401324272155762
训练次数:3100 , Loss: 1.6196000576019287
训练次数:3200 , Loss: 1.4714313745498657
本epoch上整体验证集上的Loss:  260.5732116699219
本epoch上整体验证集上的Acc  :  0.4
-----------------第2轮训练开始------------------
训练次数:3300 , Loss: 1.6874343156814575
训练次数:3400 , Loss: 1.464526653289795
训练次数:3500 , Loss: 1.3847261667251587
训练次数:3600 , Loss: 1.3109409809112549
训练次数:3700 , Loss: 1.4337337017059326
训练次数:3800 , Loss: 1.500367522239685
训练次数:3900 , Loss: 1.365796685218811
训练次数:4000 , Loss: 1.2445425987243652
本epoch上整体验证集上的Loss:  249.33676147460938
本epoch上整体验证集上的Acc  :  0.4285
-----------------第3轮训练开始------------------
训练次数:4100 , Loss: 1.4116584062576294
训练次数:4200 , Loss: 1.177223563194275
训练次数:4300 , Loss: 1.6078356504440308
训练次数:4400 , Loss: 1.4061129093170166
训练次

### torch.argmax方法
---
输出逐行(或者逐列)最大值的索引号，放在一个行向量中，输出结果形如：[x , y, z , q, m, n]，可以根据下面的原理来优化训练过程中的一些可视化数据，具体用途是可以根据这个做出`acc`。


In [18]:
import torch

out_put = torch.tensor([[0.1, 0.2],
                                                  [0.05, 0.4],])

print(out_put.argmax(1)) # 参数为1的时候是横向比较（逐行比较）

tensor([1, 1])


In [19]:
print(out_put.argmax(0)) # 参数为0的时候是纵向比较（逐列比较）0.1 > 0.05 => 索引0更大 => 输出0 ;  0.2 < 0.4 => 索引1更大 => 输出1 ======> 最后结果是[0,  1]

tensor([0, 1])


In [21]:
preds = out_put.argmax(1)
goal = torch.tensor([0, 1])
print(preds == goal)

tensor([False,  True])


👇可以输出预测值和目标值匹配的个数

In [22]:
print((preds == goal).sum())

tensor(1)
